In [ ]:
"""
Author: Liam Bogucki
Email: lboguck@uwo.ca
First Written: Friday, July 5, 2024
Last Modified: Tuesday, August 12, 2025
Program Purpose: To create the MODIS map that is partioned how Ahlstrohm did it. This file creates both the save file to be used for later...
iterations using this defintion, as well as the map of what Ahlstrohm's defintion looks like based on updated MODIS and temperature dataset.
"""

In [ ]:
# Importing appropriate libraries.
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.colors import ListedColormap
import matplotlib.patches as mpatches
import cartopy.crs as ccrs
import cartopy.feature as cfeature
# Changing over the font
plt.rcParams['font.family'] = 'Times New Roman'

In [ ]:
#This block handles the creation of a numpy array reflective of the regions as defined by Ahlstorhm

#Loading temp data
temp_data = np.load("1=tropical_0=not_monthly_temps_1940-now.npy")

#Loading current MODIS data
data_modis = np.load("Modis_0.5Deg_Type3_Mode.npy")

#Making the new map with the correct aggregations of MODIS classes to be reflective of Ahlstrohms partition
global_map = np.zeros((360,720))
#Tropical forest == 1
global_map[((data_modis == 5) | (data_modis == 6) | (data_modis == 7) | (data_modis == 8)) & ((temp_data == 1))] = 1 
#Extra-tropical forest == 2
global_map[((data_modis == 5) | (data_modis == 6) | (data_modis == 7) | (data_modis == 8)) & ((temp_data == 0))] = 2
#Semi-arid == 3
semi_arid_mask = np.zeros((360,720))
semi_arid_mask[(data_modis == 2) | (data_modis == 4)] = 1
semi_arid_mask[:90, :] = 0 #This is the 45 degree cutoff for the semi arid region but it is half-deg resolution
global_map[semi_arid_mask == 1] = 3
#Arctic-Shrub == 4
acrtic_shrub_mask = np.zeros((360,720))
acrtic_shrub_mask[(data_modis == 2) | (data_modis == 4)] = 1
acrtic_shrub_mask[90:, :] = 0 #This is the 45 degree cutoff for the arctic shrub region but it is half-deg resolution
global_map[acrtic_shrub_mask == 1] = 4
#Grassland & Crops == 5
global_map[((data_modis == 1) | (data_modis == 3))] = 5
#Sparsely vegetated == 6
global_map[(data_modis == 9) | (data_modis == 10)] = 6

#Verifying the regions are correctly created to reflect what Ahlstrohm's partition specified
plt.subplots(figsize=(20,10))
plt.imshow(global_map, cmap=ListedColormap(colors = ['white', 'limegreen', 'darkgreen', 'darkorange', 'grey', 'slateblue', 'white']), interpolation = 'none')
plt.colorbar()

#Saving the array to be used later
np.save("MODIS_Map.npy", global_map)



In [ ]:
#This block of code handles the generation of the nice map of what the regions look like

#Loading the map created above
modis_map = np.load("MODIS_Map.npy")

#Plot the map
fig = plt.figure(figsize=(40, 20))
ax = plt.axes(projection=ccrs.PlateCarree())

# Adding the appropriate features to the map
ax.add_feature(cfeature.LAND)
ax.add_feature(cfeature.OCEAN)
ax.add_feature(cfeature.COASTLINE)
ax.add_feature(cfeature.BORDERS, linestyle=':')

# Plotting the global map
ax.imshow(modis_map, extent= [-180, 180, -90, 90], 
          cmap=ListedColormap(colors = ['white', 'limegreen', 'darkgreen', 'darkorange', 'grey', 'slateblue', 'white']),
     transform=ccrs.PlateCarree(), interpolation='none')
ax.set_extent([-180, 180, -60, 90], crs=ccrs.PlateCarree()) #Setting the extent of the map such that antarctica is excluded

#Adding the legend
tropicalBar = mpatches.Patch(facecolor='limegreen', label='Tropical Forest', edgecolor='black')
extraTropicalBar = mpatches.Patch(facecolor='darkgreen', label='Extra-Tropical Forest', edgecolor='black')
semiAridlBar = mpatches.Patch(facecolor='darkorange', label='Semi-Arid', edgecolor='black')
arcticShrubBar = mpatches.Patch(facecolor='grey', label='Arctic Shrub & Tundra', edgecolor='black')
grassBar = mpatches.Patch(facecolor='slateblue', label='Crops & Grasslands', edgecolor='black')
sparseBar = mpatches.Patch(facecolor='white', label='Sparsely Vegetated', edgecolor='black')
plt.legend(handles=[tropicalBar, extraTropicalBar, semiAridlBar, arcticShrubBar, grassBar, sparseBar ], loc='lower left', fontsize=45, framealpha=0.5)

#Title
plt.title("(a)", fontsize=70, loc='left', fontweight='bold')

#Show
plt.show()

